#Integrantes
* Raúl Morales Galindo
* Hector Suarez Camacho
* Fernando Mateo Valencia Gomez

# Dataset

In [99]:
import folium
import pandas as pd
import math
import heapq
import numpy as np

#cities = pd.read_csv("/content/cities_cord.csv")
#cities = cities[['city','lat','lng','country','admin_name', 'capital']]
cities_url ='https://raw.githubusercontent.com/Rcgil30/Lab2ED2/main/data/cities_cord.csv?token=GHSAT0AAAAAABYNZVOT5XKRGNAMUEOHM3VOYZOGQ4A'
cities = pd.read_csv(cities_url)
cities  = cities[(cities['country'] == "Colombia")]
cities  = cities[(cities['capital'] == "admin") | (cities['capital'] == "primary")]
cities['city'] = cities['city'].str.upper()
cities = cities[['city','lat','lng']]

In [ ]:
cities

# Distancia a partir de coordenadas

In [101]:
def haversine_distance(lat1, lon1, lat2, lon2, r=6371):
    coordinates = lat1, lon1, lat2, lon2
    phi1, lambda1, phi2, lambda2 = [
        math.radians(c) for c in coordinates
    ]  
    a = (np.square(math.sin((phi2-phi1)/2)) + math.cos(phi1) * math.cos(phi2) * 
         np.square(math.sin((lambda2-lambda1)/2)))
    d = 2*r*math.asin(np.sqrt(a))
    return d

# Grafo

In [102]:
class Node:
    def __init__(self, data: str) -> None:
        # Nombre de la ciudad
        self.data = data
        # Latitud y longitud de la ciudad
        self.lat: float = 0
        self.lng: float = 0
        self.connections: list[Node] = []

class Grafo:
    def __init__(self) -> None:
        # Cada elemento de la lista de vertices corresponde en su index a la de ciudades
        self.listavertices: list[Node] = []
        self.listaciudades: list[str] = []
        self.MatrizDis: list[list[float]] = []

In [103]:
grafo = Grafo()
"""Iteramos a través del df y añadimos las ciudades al grafo"""
c = 0
for index, cap in cities.iterrows():
    ciudad = cap["city"]
    """Para cada ciudad que no este incluida en el grafo, definimos
       sus atributos y la incluimos a este
    """
    if ciudad not in grafo.listaciudades:
        grafo.listaciudades.append(ciudad)
        nodo = Node(ciudad)
        grafo.listavertices.append(nodo)
        nodo.pos = c
        nodo.lat = float(cap["lat"])
        nodo.lng = float(cap["lng"])
        c += 1

for i in range(len(grafo.listavertices)):
            row = [0.0] * len(grafo.listavertices)
            grafo.MatrizDis.append(row)


In [104]:
# Matriz de distancia
for i in range(len(grafo.listavertices)):
    for j in range(i+1, len(grafo.listavertices)):
        nodo1 = grafo.listavertices[i]
        nodo2 = grafo.listavertices[j]
        distancia = haversine_distance(nodo1.lat, nodo1.lng, nodo2.lat, nodo2.lng)
        nodo1.connections.append(nodo2)
        nodo2.connections.append(nodo1)
        grafo.MatrizDis[i][j] = distancia
        grafo.MatrizDis[j][i] = distancia



#Nodos

In [ ]:
mapG = folium.Map(location=[4.570868,-74.297333],zoom_start=4)
for index, location_info in cities.iterrows():
    folium.Marker([location_info["lat"], location_info["lng"]], popup=location_info["city"], icon=folium.Icon(color="blue", icon="plane")).add_to(mapG)
mapG

# Creación de aristas

In [ ]:
# Crear una lista vacía para almacenar todos los objetos PolyLine
polylines = []

# Iterar sobre cada par de nodos en el grafo y crear una PolyLine para cada arista
for i in range(len(grafo.listavertices)):
    for j in range(i+1, len(grafo.listavertices)):
        nodo1 = grafo.listavertices[i]
        nodo2 = grafo.listavertices[j]
        # Obtener el peso de la arista
        weight = round(grafo.MatrizDis[i][j], 2)
        # Crear una PolyLine utilizando las coordenadas de los dos nodos
        polyline = folium.PolyLine(locations=[(nodo1.lat, nodo1.lng), (nodo2.lat, nodo2.lng)], color='purple', tooltip=str(weight)+' km')
        # Agregar la PolyLine a la lista de polylines
        polylines.append(polyline)
        #print(nodo2.lng)
# Agregar todas las PolyLines al mapa al mismo tiempo
for polyline in polylines:
    polyline.add_to(mapG)


#Mapa conectado

In [ ]:
mapG

# Algoritmo Prim

In [108]:
def prim(graph):
    # Inicializar el árbol de expansión mínima y el conjunto de nodos visitados
    mst = []
    visited = set()
    
    # Elegir un nodo inicial arbitrario y agregarlo al conjunto de nodos visitados
    start_node = 0
    visited.add(start_node)
    
    # Inicializar la cola de prioridad con las aristas adyacentes al nodo inicial
    edges = [(weight, start_node, end_node) for end_node, weight in enumerate(graph[start_node])]
    heapq.heapify(edges)

    # Continuar mientras la cola de prioridad no esté vacía
    while edges:
        # Obtener la arista con el menor peso
        weight, start_node, end_node = heapq.heappop(edges)
        
        # Si el nodo final no ha sido visitado, agregar la arista al árbol de expansión mínima
        if end_node not in visited:
            visited.add(end_node)
            mst.append((start_node, end_node))
            
            # Agregar las aristas adyacentes al nodo final a la cola de prioridad
            for next_end_node, weight in enumerate(graph[end_node]):
                if next_end_node not in visited:
                    heapq.heappush(edges, (weight, end_node, next_end_node))
    
    return mst

In [109]:
def calcular_peso_total(mst, graph):
    peso_total = 0
    for start_node, end_node in mst:
        peso_total += graph[start_node][end_node]
    return peso_total


In [110]:
def dibujar_aristas_mst(map, grafo, mst):
    # Crear una lista vacía para almacenar todas las PolyLines
    polylines = []
    
    # Iterar sobre cada arista en el árbol de expansión mínima
    for start_node, end_node in mst:
        # Obtener los objetos Node correspondientes a los nodos inicial y final de la arista
        nodo1 = grafo.listavertices[start_node]
        nodo2 = grafo.listavertices[end_node]
        
        # Obtener el peso de la arista
        weight = round(grafo.MatrizDis[start_node][end_node], 2)

        # Crear una PolyLine utilizando las coordenadas de los dos nodos
        polyline = folium.PolyLine(locations=[(nodo1.lat, nodo1.lng), (nodo2.lat, nodo2.lng)], color='red', tooltip=str(weight)+' km')
        
        # Agregar la PolyLine a la lista de polylines
        polylines.append(polyline)
    
    # Agregar todas las PolyLines al mapa al mismo tiempo
    for polyline in polylines:
        polyline.add_to(map_prim)


In [111]:
# Calcular el árbol de expansión mínima del grafo utilizando el método kruskal
mstp = prim(grafo.MatrizDis)

#peso del arbol
peso_total = calcular_peso_total(mstp, grafo.MatrizDis)

# Crear un nuevo objeto map de folium
map_prim = folium.Map(location=[4.570868,-74.297333],zoom_start=6)
for index, location_info in cities.iterrows():
    folium.Marker([location_info["lat"], location_info["lng"]], popup=location_info["city"], icon=folium.Icon(color="blue", icon="plane")).add_to(map_prim)

# Dibujar solo las aristas del árbol de expansión mínima en el nuevo mapa utilizando el método dibujar_aristas_mst
dibujar_aristas_mst(map_prim, grafo, mstp)

# Mapa de Prim

In [ ]:
print(f'El peso del arbol de expansión minima por prim tiene un peso de: {round(peso_total,2)} en km')
map_prim

# Algoritmo Kruskal

In [113]:
# Crear un nuevo objeto map de folium
map_kruskal = folium.Map(location=[4.570868,-74.297333],zoom_start=6)
for index, location_info in cities.iterrows():
    folium.Marker([location_info["lat"], location_info["lng"]], popup=location_info["city"], icon=folium.Icon(color="blue", icon="plane")).add_to(map_kruskal)


In [114]:
def kruskal(graph):
    # Inicializar el árbol de expansión mínima y el conjunto de nodos visitados
    mst = []
    visited = set()
    
    # Crear una lista de todas las aristas del grafo
    edges = []
    for start_node in range(len(graph)):
        for end_node, weight in enumerate(graph[start_node]):
            edges.append((weight, start_node, end_node))
    
    # Ordenar las aristas por peso
    edges.sort()
    
    # Inicializar un arreglo para almacenar los conjuntos disjuntos de nodos
    parent = [i for i in range(len(graph))]
    
    # Función auxiliar para encontrar el conjunto al que pertenece un nodo
    def find_parent(node):
        if parent[node] == node:
            return node
        parent[node] = find_parent(parent[node])
        return parent[node]
    
    # Función auxiliar para unir dos conjuntos disjuntos
    def union(u, v):
        u_set = find_parent(u)
        v_set = find_parent(v)
        parent[u_set] = v_set
    
    # Iterar sobre cada arista en orden ascendente por peso
    for edge in edges:
        weight, start_node, end_node = edge
        
        # Si los nodos inicial y final de la arista pertenecen a conjuntos disjuntos,
        # agregar la arista al árbol de expansión mínima y unir los conjuntos
        if find_parent(start_node) != find_parent(end_node):
            union(start_node, end_node)
            mst.append((start_node, end_node))
    
    return mst

# Calcular el árbol de expansión mínima del grafo utilizando el método kruskal
mst = kruskal(grafo.MatrizDis)

# Calcular el costo total del árbol de expansión mínima
costo_totalK = 0
for start_node, end_node in mst:
    costo_totalK += grafo.MatrizDis[start_node][end_node]

In [115]:
def dibujar_aristas_mst(map, grafo, mst):
    # Crear una lista vacía para almacenar todas las PolyLines
    polylines = []
    
    # Iterar sobre cada arista en el árbol de expansión mínima
    for start_node, end_node in mst:
        # Obtener los objetos Node correspondientes a los nodos inicial y final de la arista
        nodo1 = grafo.listavertices[start_node]
        nodo2 = grafo.listavertices[end_node]
        
        # Obtener el peso de la arista
        weight = round(grafo.MatrizDis[start_node][end_node], 2)

        # Crear una PolyLine utilizando las coordenadas de los dos nodos
        polyline = folium.PolyLine(locations=[(nodo1.lat, nodo1.lng), (nodo2.lat, nodo2.lng)], color='green', tooltip=str(weight)+' km')
        
        # Agregar la PolyLine a la lista de polylines
        polylines.append(polyline)
    
    # Agregar todas las PolyLines al mapa al mismo tiempo
    for polyline in polylines:
        polyline.add_to(map_kruskal)

In [116]:
# Calcular el árbol de expansión mínima del grafo utilizando el método kruskal
mstk = kruskal(grafo.MatrizDis)

# Crear un nuevo objeto map de folium
map_kruskal = folium.Map(location=[4.570868,-74.297333],zoom_start=6)

# Agregar marcadores para cada ciudad en el mapa
for index, location_info in cities.iterrows():
    folium.Marker([location_info["lat"], location_info["lng"]], popup=location_info["city"], icon=folium.Icon(color="blue", icon="plane")).add_to(map_kruskal)

# Dibujar solo las aristas del árbol de expansión mínima en el nuevo mapa utilizando el método dibujar_aristas_mst
dibujar_aristas_mst(map_kruskal, grafo, mstk)

# Mapa de Kruskal

In [ ]:
print(f'El peso del arbol de expansión minima por Kruskal tiene un peso de: {round(costo_totalK,2)} en km')
map_kruskal